In [1]:
class Book:
    def __init__(self, title, price):
        self.title = title
        self.price = price
    
class Cart:
    def __init__(self):
#         self.items will store Book objects
        self.items = []
    
    def total_price(self):
        sumPrice = 0
        for item in self.items:
            sumPrice += item.price
        
        return sumPrice
    
class PrintedBook(Book):
    def __init__(self, title, price, weight):
        super().__init__(title, price)
        self.weight = weight
        
class VirtualBook(Book):
    def __init__(self, title, price, downloadLink):
        super().__init__(title, price)
        self.download_link = downloadLink


In [3]:
with open("bookstore.txt") as fs:
    data = fs.read().strip("\n").split("\n")
    for item in data:
        row = item.split(",")
        if(row[2] == "Physical"):
            newPrinted = PrintedBook(row[0], float(row[1]), int(row[3]))
        
        elif(row[2] == "Virtual"):
            newVirtual = VirtualBook(row[0], float(row[1]), row[4])

In [9]:
import sqlite3

client = sqlite3.connect("bookstore.db")

with open("bookstore.txt") as fs:
    data = fs.read().strip("\n").split("\n")
    for item in data:
        row = item.split(",")
        if(row[2] == "Physical"):
            nBook = PrintedBook(row[0], float(row[1]), int(row[3]))
            client.execute("INSERT INTO Book(Title,Price,Type) VALUES (?,?,?)", (nBook.title, nBook.price, "Physical"))
            curr = client.execute("SELECT BookID FROM Book WHERE Title = ?", (nBook.title,))
            BookID = list(curr)[0][0]
            client.execute("INSERT INTO Printed(BookID, Weight) VALUES (?,?)", (BookID, nBook.weight))
            
        
        elif(row[2] == "Virtual"):
            nBook = VirtualBook(row[0], float(row[1]), row[4])
            client.execute("INSERT INTO Book(Title,Price,Type) VALUES (?,?,?)", (nBook.title, nBook.price, "Virtual"))
            curr = client.execute("SELECT BookID FROM Book WHERE Title = ?", (nBook.title,))
            BookID = list(curr)[0][0]
            client.execute("INSERT INTO Virtual(BookID, DownloadLink) VALUES (?,?)", (BookID, nBook.download_link))
         
client.commit()
client.close()